In [ ]:
import os
import sys
import pandas as pd
import numpy as np

dataset_dir = "../../datasets/airbus-ship-detection"

annotations = pd.read_csv(dataset_dir + '/train_ship_segmentations_v2.csv')
        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
ship_anns = annotations[~annotations.EncodedPixels.isna()]

In [ ]:
image_names = ship_anns.ImageId.unique().tolist()

In [ ]:
print(image_names[3])

In [ ]:
image_annotation = ship_anns.query('ImageId=="' + image_names[3] + '"')['EncodedPixels']

In [ ]:
print(image_annotation)

In [ ]:
len(image_names)

In [ ]:
len(ship_anns)

In [ ]:
len(annotations)

In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [ ]:
rle_decode(annotations['EncodedPixels'][3])

In [ ]:
        # Add images
        iimageCount = 0
        for a in annotations:            
            polygons = anns.query('ImageId=="' + fp + '"')['EncodedPixels']

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            
            if ( (subset == "train" and iimageCount < 60) or (subset == "val" and iimageCount >= 60) ):
                self.add_image(
                    "airbus",
                    image_id=a['filename'],  # use file name as a unique image id
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons)
            iimageCount+=1


In [1]:
from mrcnn import model as modellib, utils
class PotholeDataset(utils.Dataset):
    def load_airbus(self, dataset_dir, subset):
        """Load a subset of the Airbus dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("ship", 1, "ship")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        annotations = pd.read_csv(dataset_dir + '/train_ship_segmentations_v2.csv')

        # skip images with no annotations
        ship_anns =  annotations[~annotations.EncodedPixels.isna()]

        # there are multiple anns for an image so we need to unique the filenames        
        image_names = ship_anns.ImageId.unique().tolist()
        
        # Add images
        iimageCount = 0
        for image_name in image_names:   
            image_annotation = ship_anns.query('ImageId=="' + image_name + '"')['EncodedPixels']
            height, width = orig_height, orig_width
            
            if ( (subset == "train" and iimageCount < 38000) or (subset == "val" and iimageCount >= 38000) ):
                self.add_image(
                    "airbus",
                    image_id=iimageCount,
                    path=image_path,
                    width=width, height=height,
                    image_annotation=image_annotation)
            iimageCount+=1
            if (iimageCount>3):
                sys.exit()
                
    def rle_decode(mask_rle, shape=(768, 768)):
        '''
        mask_rle: run-length as string formated (start length)
        shape: (height,width) of array to return 
        Returns numpy array, 1 - mask, 0 - background
        '''
        s = mask_rle.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
        return img.reshape(shape).T  # Needed to align to RLE direction

                
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a airbus dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "airbus":  ##correct 
            return super(self.__class__, self).load_mask(image_id)

        # take the annotation for the image and decode from the RLE
        annotations = info['annotations']
        print(image_id, annotations)
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            for i, a in enumerate(annotations):
                mask[:, :, i] = rle_decode(a)

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)


Using TensorFlow backend.


ImportError: cannot import name '_validate_lengths'

In [ ]:
load_airbus(dataset_dir, "train")